In [88]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd

In [89]:
driver = webdriver.Chrome('./chromedriver.exe')
df = pd.read_csv('./data/tour_data/tour_data.csv')
df1 = df.loc[:,['tour_id','title']]

In [90]:
title_list = []
tour_id_list = []
overview_list = []

In [91]:
def main():
    global driver
    
    driver.implicitly_wait(3)
    driver.get('https://map.kakao.com/')
    
    for i in range(220, 230): # 범위 설정 **** 변경 ****
        sleep(2)
        print('####', i+1)
        search(df1['title'][i], i+1)
    driver.quit()
    print("finish")

In [92]:
#카카오맵 첫화면에서 관광지명으로 검색한뒤 첫번째 결과로 crawilng 함수 호출하는 함수
def search(place, tour_id):
    global driver
    
    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)
    html = driver.page_source
    
    try :
        soup = BeautifulSoup(html, 'html.parser')
        place= soup.select('.placelist > .PlaceItem')[0] # 검색된 장소 목록
        try :
            sleep(1)
            crawling(place, tour_id)
        
        except ElementNotInteractableException:
            print('not found')
        finally:
            search_area.claer()
    except:
        print('no search result')
        search_area.clear()

In [93]:
def crawling(place, tour_id):
    place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
    detail_page_xpath = '//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]'
    driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
    driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
    sleep(1)
    
    print('####', place_name, tour_id)
    
    # 첫 페이지
    extract_overview(place_name, tour_id)
    
    driver.close()
    driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환

In [94]:
def extract_overview(place_name, tour_id):
    global driver
    
    ret = True
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 개요 찾기
    overview = soup.select('.txt_introduce')
    
    # 개요가 있는 경우
    if len(overview) != 0 :
        title_list.append(place_name)
        tour_id_list.append(tour_id)
        overview_list.append(overview[0].text)
        print(tour_id, overview)
    
    else :
        print('no overreview in extract')
        title_list.append(place_name)
        tour_id_list.append(tour_id)
        overview = None
        overview_list.append(overview)
        ret = False
    return ret

In [95]:
if __name__ == "__main__":
    main()

#### 221
#### 밥보다국시 221
no overreview in extract
no search result
#### 222
#### 국제갤러리 222
222 [<p class="txt_introduce">
                    세계적으로 유명한 현대미술 작가의 작품과 그 흐름을 접하고 감상할 수 있는 곳
                </p>]
no search result
#### 223
#### 국제선센터 223
223 [<p class="txt_introduce">
                    1700년의 역사를 지닌 한국불교의 전통과 문화, 특히 간화선을 전 세계에 알리기 위해 건립된 템플스테이센터.
                </p>]
no search result
#### 224
no search result
#### 225
#### 국제 자수원 3호점 225
no overreview in extract
no search result
#### 226
#### 국제청소년센터 드림텔 226
no overreview in extract
no search result
#### 227
#### 국화정원 227
no overreview in extract
no search result
#### 228
#### 국회의사당 헌정기념관(휴관중) 228
228 [<p class="txt_introduce">
                    헌정기념관은 1998년 5월 29일 국회개원 50주년 기념사업의 일환으로 건립, 개관하여 국민에게 개방해오고 있다.
                </p>]
no search result
#### 229
#### 국회의사당 229
229 [<p class="txt_introduce">
                    여성가족위원회,국회사무처,통일외교통상위원회,법제사법위원회,보건복지위원회,문화관광위원회,국방위원회
                </p>]
no search result
#### 230

In [99]:
pd.DataFrame({'tour_id' :tour_id_list, 'overview' : overview_list})

,tour_id,overview
0,221,None
1,222,\n 세계적으로 유명한 현대미술 작가의 작품과 그...
2,223,\n 1700년의 역사를 지닌 한국불교의 전통과 ...
3,225,None
4,226,None
5,227,None
6,228,\n 헌정기념관은 1998년 5월 29일 국회개원...
7,229,"\n 여성가족위원회,국회사무처,통일외교통상위원회,..."
8,230,None
